> Including motorstate unique Brands 412, 461, 404, 425, 342, 132, 414, 337, 233, 481, 850, 867, 479, 690, 579 in price files

> Activating Wheel Pros (putting 0 as Ship Cost)

> Wheel Pros on BS Amazon - WP Shipping file correction

> Bundle SKU Remove Duplicates enhancement

> Turn 14

> Black Burn inclusion in price files (shipping set as 0)

> WTD Arroyo Shipping Estimates

> DGTire Shipping Estimates

> Inclusion of My Motor Madness

> Inclusion of Dorman Manual Stock update

> Inclusion of Tonsa Fedex based shipping Estimates

In [1]:
import pickle
from os.path import exists
import warnings
import time
warnings.filterwarnings(action='once')
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import math
pd.options.display.max_columns = 999

start_time = time.time()

## Setup Stuff (that should come from config files or database later)

CHANNEL: Setting up Channel Transaction Fee (%) for each Channel >> List of Dictionaries

WAREHOUSES: Defining Keys and Shipping Charge Category for Each WareHouse >> List of Dictionaries

get_warehouse_key(warehouse_name): Return Key of the given Warehouse (from the WAREHOUSES) - can be improved

Initializing DEFAULT_TARGET_PROFIT(5%), DEFAULT_SHIP_MARKUP (12%), EXCLUDED_WAREHOUSES, PUNCTUATION_WAREHOUSES (Punctuation has to be retained in these)

PARTS_AUTH_SHIPPING_MODEL: The Machine pretrained Machine Learning Model loaded from pickle
PRICE_FILE_COLUMNS: Columns in the Price File

PRICE_FILE_LOCATION
CatSKU_CHANNELS // Need elaboration


In [2]:
CS_TRANSACTION_FEE = 0.0101

CHANNELS = [
    {'name':'AP Fusion', 'id':'APF'}, #Changed on 7th Oct
    {'name':'PS Amazon', 'id':'PSA'},
    {'name':'PS Walmart', 'id':'PSW'},
    {'name':'PS Ebay', 'id':'PSE'},  ## Changed back to 0.12 here. The 0.02 for WHI will be added added separately for CatSKUs
    {'name':'BS Amazon', 'id':'BSA'},
    {'name':'BS Walmart', 'id':'BSW'},
    {'name':'BS Ebay', 'id':'BSE'},
    {'name':'Mecka', 'id':'BAABS'}, 
    {'name':'My Motor Madness', 'id':'MMM'} 
]

WAREHOUSES = [
    # Fully integrated warehouses
    {'key':'C', 'name':'Brock', 'shipping':'free-ish'},
    {'key':'D', 'name':'Dorman Direct', 'shipping':'free-ish'},
    {'key':'J', 'name':'PFG', 'shipping':'theirs'},
    {'key':'K', 'name':'Keystone', 'shipping':'theirs'},
    {'key':'N', 'name':'NPW', 'shipping':'ours'},
    {'key':'O', 'name':'Tonsa', 'shipping':'ours'},
    {'key':'P', 'name':'Parts Auth', 'shipping':'theirs'},
    {'key':'Y', 'name':'Motor State', 'shipping':'theirs'},
    # Manual/FTP warehouses
    {'key':'1', 'name':'Jante Wheel', 'shipping':'free'},
    {'key':'2', 'name':'OE Wheels', 'shipping':'theirs'},
    {'key':'6', 'name':'Burco Mirrors', 'shipping':'ours'},
    {'key':'8', 'name':'Race Sport Lighting', 'shipping':'ours', 'target_profit': 0.1},
    {'key':'9', 'name':'Sunbelt APG', 'shipping':'ours'},
    {'key':'Z', 'name':'SimpleTire', 'shipping':'free'}
    # Low-volume, or unused warehouses
    #{'key':'5', 'name':'KSI Trading'},
    #{'key':'7', 'name':'NTW'},
    #{'key':'H', 'name':'Hanson'},
    #{'key':'3', 'name':'Motor State'},
]

def get_warehouse_key(warehouse_name):
    for warehouse in WAREHOUSES:
        if warehouse['name'] == warehouse_name:
            return warehouse['key']
    return None


DEFAULT_TARGET_PROFIT = 0.05
DEFAULT_SHIP_MARKUP = 1 / 1.12
EXCLUDED_WAREHOUSES = ['A','H', 'C']
#EXCLUDED_WAREHOUSES = ['A','5','H','3', '7', '4', 'C', '3']
PUNCTUATION_WAREHOUSES = ['J','1','C', '9', '8', 'Y', 'F', '7', 'U', '6', '4']

PARTS_AUTH_SHIPPING_MODEL = 'shipping-research/tree-model.pkl'

PRICE_FILE_COLUMNS = ['CS-SKU-NP', 'MinPrice', 'Shipping', 'Carrier', 'Service', 'Markup',
       'ShipMkup', 'ListMkup', 'PackQty', 'MinQty', 'MaxQty', 'Zip Code',
       'CatSKU', 'OP-Lowest(Y)', 'VND-Lowest(Y)', 'MinMkDown', 'MaxMkUp', 'Interval',
       'BundleSKU', 'Duplicate']

PRICE_FILE_LOCATION = 'price-files'

#CatSKU_CHANNELS = ['PS Ebay']

# Main

#### Load in data.

Reading the Price Weight file from Disk

warehouses: DataFrame containing unique warehouse keys

In [3]:
pw = pd.read_csv('inventory/pw-all.csv', low_memory=False, 
                 dtype={'MasterLC':'Int64', 'Zip Code': str})
# pw['MasterLC'] = pw['MasterLC'].astype('Int64')

# Temporarily remove all NPW.
##pw = pw[pw['WD'] != 'N']

warehouses = pw['WD'].unique()

## Top-level processing and filtering.

#### Sad updates.

In the Priceweight data (pw) copying CS-SKU to CS-SKU-NP - basically copying the SKU with punctuation into the non-punctuation SKU column (for Brock 'C')\
Setting MasterLC to 429 if MasterLC is 158 and 429


In [4]:
# Correct the Line Code and CS-SKU-NP for Brock

pw.loc[(pw['MasterLC']==158) & (pw['LC']=='429'), 'MasterLC'] = 429
pw.loc[(pw['WD']=='C'), 'MasterSKU'] = pw.loc[(pw['WD']=='C'), 'MasterSKU'].str.replace('158|','429|', regex=False)
pw.loc[(pw['WD']=='C'), 'CS-SKU'] = pw.loc[(pw['WD']=='C'), 'CS-SKU'].str.replace('158|','429|', regex=False)
# pw.loc[pw['WD']=='C', 'CS-SKU-NP'] = pw.loc[pw['WD']=='C', 'CS-SKU']

#### Preprocess price file.

Reading MAP (Minimum Advertized Price) from file against each SKU and dropping duplicates if any\
Sorting on 'CS-SKU-NP', dropping duplicates based on CS-SKU-NP and setting CS-SKU-NP as index -- This creates a unique sorted index which has performance benefits https://stackoverflow.com/questions/16626058/what-is-the-performance-impact-of-non-unique-indexes-in-pandas \
Basically, it can search any value in O(1) time\
In the end, we get a Pandas Series with SKU as index and MAP as value

In [5]:
map_prices = pd.read_csv('maps.csv')
map_prices = map_prices.sort_values('CS-SKU-NP').drop_duplicates(subset=['CS-SKU-NP'])
map_prices = map_prices.set_index('CS-SKU-NP')['MAP']

Creating a new column 'item_cost' by copying 'MinPrice'\
Creating filter 'dorman_update_idx'=> where Warehouse is Dorman ('D') and PackQty is not na
Using this filter to convert prices from per pack to per unit

Creating backup copy of cs-sku-np with name cs-sku-np-catsku

For Punctuation warehouses, Create CS-SKU-NP by concatenating Key, MasterLC, | and Part Number\
for non-Puncuation warehouses, remove non-alphanumeric characters from Part Number - This temporary step of copying to temporary variable 'x' can be avoided by filtering and assigning using loc function directly
lambda operator defines a function in a single line - this function is passed as a filter to map

In the Price Weight dataframe (pw), adding a new column 'MAP' (Minimum Advertized Price) with MAP prices from above step if available and 1 if not available in above data (map_prices)\

Removing WeatherTech (310) from Price Weight File

Set not available values indicater (9999) back to actual nan

In [6]:
# Define separate column for managing costs vs MinPrice to avoid confusion.
pw['item_cost'] = pw['MinPrice']

# update Dorman costs (which are per-pack initally) to be per-unit
# dorman_update_idx = (pw['WD']=='D') & (pw['PackQty'].notna())
# pw.loc[dorman_update_idx, 'item_cost'] = pw.loc[dorman_update_idx, 'item_cost'] / pw.loc[dorman_update_idx, 'PackQty']
# del dorman_update_idx
pw.loc[pw['WD']=='D', 'PackQty'] = 1   #setting the PackQty for items from Dorman warehouse to 1 because they give us the price of the whole package - no need for Pack quantity multiplication

pw['CS-SKU-NP'] = pw['WD-SKU']
#Set CSSKUNP depending on if it's a punctuation warehouse
x = pw[pw['WD'].isin(PUNCTUATION_WAREHOUSES)].copy()
x['CS-SKU-NP'] = x['WD'] + x['MasterLC'].astype(str) + '|' + x['Part Number']
pw.loc[pw['WD'].isin(PUNCTUATION_WAREHOUSES), :] = x

pw = pw.loc[~pw['Part Number'].isnull()] #Removing null Part Numbers

x = pw[~pw['WD'].isin(PUNCTUATION_WAREHOUSES)].copy()
x['CS-SKU-NP'] = (x['WD'] + x['MasterLC'].astype(str) + '|' 
                  + x['Part Number'].map(lambda s: ''.join(filter(str.isalnum, s))))
pw.loc[~pw['WD'].isin(PUNCTUATION_WAREHOUSES), :] = x

pw = pw.join(other=map_prices, on='CS-SKU-NP', how='left')   #optimized code for fetching MAP columnt from map_prices
pw['MAP'].fillna(1, inplace=True)
del map_prices   #This dataset is no longer needed

# Remove WeatherTech (just in case)
pw = pw[pw['MasterLC'] != 310]

# Remove First Stop Brakes Dorman Line
# df = df[~((df['WD']=='D') & df['Part Number'].isin(first_stop_brakes))]
# Nope, actually don't remove them, just set MinQty really high... at the end.

# Remove placeholder values for Weight/ShipWeight
pw.loc[(pw['Weight']==9999), 'Weight'] = np.nan
pw.loc[(pw['ShipWeight']==9999), 'ShipWeight'] = np.nan

pw['Shipping_Add'] = 0.0 #Creating a column for additional Shipping
pw_cols = pw.columns   #columns in the Price Weight Report

In [7]:
#Dorman Items pack Quantity Adjustment

pw.loc[pw['MasterSKU']=='591|611034', 'PackQty'] = 1

#### Filter parts.

Remove Excluded warehouses from price weight data\
Adjusting invalid PackQty - na => 1\
Considering only values with PackQty <=10 or any Dorman values (we have clean data for Dorman)

In [8]:
# Idea here is to filter out all the lil nasties that we don't want to include.
# This could differ by warehouse, or not.
# Things like, heavy parts, big or oddly shaped parts, 
# parts that are really expensive, or come in packs of many.
# ... See notes on original Jim conversation for what all you should be including here.

# Filter out excluded warehouses.
pw = pw[~pw['WD'].isin(EXCLUDED_WAREHOUSES)]
# Filter out nasty pack quantities. (allow these for Dorman, since we have clean data.)
pw['PackQty'] = pw['PackQty'].fillna(1) # assume PackQty of NA => PackQty=1
pw = pw[(pw['PackQty'] <= 10) | (pw['WD']=='D')]


# Manual Inventories

#### Brock Manual Inventory

In [9]:
#pw1 = pw.copy()

In [10]:
'''
brock = pd.read_csv('inventories/brock_20220826.csv', low_memory=False)
brock['CS-SKU'] = 'C' + brock['sku_filtered_cs']
brock = brock[['CS-SKU', 'prices', 'total_stock']]

#Merging with the Brock inventory
pw = pw.merge(brock, how='left', left_on='CS-SKU-NP', right_on='CS-SKU', suffixes=('','_b') )

pw.loc[~pw['prices'].isnull(), 'MinPrice'] = pw.loc[~pw['prices'].isnull(), 'prices']
pw.loc[~pw['prices'].isnull(), 'item_cost'] = pw.loc[~pw['prices'].isnull(), 'prices']

#updating stock from the brock inventory
pw.loc[~pw['prices'].isnull(), 'Qty'] = pw.loc[~pw['prices'].isnull(), 'total_stock']

pw.loc[((pw['prices'].isnull()) & (pw['WD']=='C')), 'Qty'] = 0  #Setting Quantity of items not present in inventory file to zero


#Restoring pw columns from before brock manual update
pw = pw[pw_cols]

del brock #Delete Brock Inventory Dataframe as it is no longer needed
'''

"\nbrock = pd.read_csv('inventories/brock_20220826.csv', low_memory=False)\nbrock['CS-SKU'] = 'C' + brock['sku_filtered_cs']\nbrock = brock[['CS-SKU', 'prices', 'total_stock']]\n\n#Merging with the Brock inventory\npw = pw.merge(brock, how='left', left_on='CS-SKU-NP', right_on='CS-SKU', suffixes=('','_b') )\n\npw.loc[~pw['prices'].isnull(), 'MinPrice'] = pw.loc[~pw['prices'].isnull(), 'prices']\npw.loc[~pw['prices'].isnull(), 'item_cost'] = pw.loc[~pw['prices'].isnull(), 'prices']\n\n#updating stock from the brock inventory\npw.loc[~pw['prices'].isnull(), 'Qty'] = pw.loc[~pw['prices'].isnull(), 'total_stock']\n\npw.loc[((pw['prices'].isnull()) & (pw['WD']=='C')), 'Qty'] = 0  #Setting Quantity of items not present in inventory file to zero\n\n\n#Restoring pw columns from before brock manual update\npw = pw[pw_cols]\n\ndel brock #Delete Brock Inventory Dataframe as it is no longer needed\n"

#### NPW Manual Inventory

In [11]:
'''
npw = pd.read_csv('inventories/npw_20220826.csv', low_memory=False)
npw['Core'].fillna(0, inplace=True)
npw['WD'] = 'N'
npw.rename(columns={'PartN':'Part Number', 'LineCode':'LC'}, inplace=True )
npw['Cost'] += npw['Core']   #Add core amount to Cost
npw.drop(columns = ['Core'], inplace=True) #Drop the Core column as it has already been added to the cost
npw['Part Number'] = npw['Part Number'].map(lambda s: ''.join(filter(str.isalnum, s)))   #Removing Punctuation from NPW Part Numbers

pw = pw.merge(npw, how='left', on = ['WD', 'LC', 'Part Number'], suffixes=('', '_npw') )   #Merging with NPW inventory
pw.loc[~pw['QA'].isnull(), 'Qty'] = pw.loc[~pw['QA'].isnull(), 'QA']   #Updating Quantity directly from NPW inventory
pw.loc[~pw['QA'].isnull(), 'MinPrice'] = pw.loc[~pw['QA'].isnull(), 'Cost']   #Updating MinPrice and Cost directly from NPW Inventory
pw.loc[~pw['QA'].isnull(), 'item_cost'] = pw.loc[~pw['QA'].isnull(), 'Cost']

pw.loc[((pw['QA'].isnull()) & (pw['WD']=='N')), 'Qty'] = 0   #Setting Quantity for NPW items not found in NPW inventory to 0

pw = pw[pw_cols]   #Restoring pw columns from before npw manual update
del npw   #Delete npw Inventory Dataframe as it is no longer needed
'''

"\nnpw = pd.read_csv('inventories/npw_20220826.csv', low_memory=False)\nnpw['Core'].fillna(0, inplace=True)\nnpw['WD'] = 'N'\nnpw.rename(columns={'PartN':'Part Number', 'LineCode':'LC'}, inplace=True )\nnpw['Cost'] += npw['Core']   #Add core amount to Cost\nnpw.drop(columns = ['Core'], inplace=True) #Drop the Core column as it has already been added to the cost\nnpw['Part Number'] = npw['Part Number'].map(lambda s: ''.join(filter(str.isalnum, s)))   #Removing Punctuation from NPW Part Numbers\n\npw = pw.merge(npw, how='left', on = ['WD', 'LC', 'Part Number'], suffixes=('', '_npw') )   #Merging with NPW inventory\npw.loc[~pw['QA'].isnull(), 'Qty'] = pw.loc[~pw['QA'].isnull(), 'QA']   #Updating Quantity directly from NPW inventory\npw.loc[~pw['QA'].isnull(), 'MinPrice'] = pw.loc[~pw['QA'].isnull(), 'Cost']   #Updating MinPrice and Cost directly from NPW Inventory\npw.loc[~pw['QA'].isnull(), 'item_cost'] = pw.loc[~pw['QA'].isnull(), 'Cost']\n\npw.loc[((pw['QA'].isnull()) & (pw['WD']=='N')

#### PFG Manual Inventory

In [12]:
'''    
pfg = pd.read_csv('inventory/pfg.txt', sep='\t', encoding_errors='ignore', escapechar='\\', low_memory=False, usecols=['SKU', 'STOCK_TOTAL', 'COST'] )
pfg['WD'] = 'J'
pfg.set_index( keys=['WD', 'SKU'], inplace=True)

pw = pw.join(other=pfg, on=['WD', 'Part Number'], how='left')   #Joining the price weight report with PFG inventory

pfg_inv_fil = (pw['WD']=='J') & (~(pw['STOCK_TOTAL'].isnull()))   #creating filter for items found in PFG inventory

#Updating Stock and Cost from PFG inventory into the price weight report
pw.loc[pfg_inv_fil, 'MinPrice'] =pw.loc[pfg_inv_fil, 'COST']
pw.loc[pfg_inv_fil, 'item_cost'] =pw.loc[pfg_inv_fil, 'COST']
pw.loc[pfg_inv_fil, 'Qty'] =pw.loc[pfg_inv_fil, 'STOCK_TOTAL']

pw.loc[(pw['WD']=='J') & (pw['STOCK_TOTAL'].isnull()), 'Qty'] = 0   #Setting the stock of PFG items not in PFG inventory to 0
pw.drop(columns=['STOCK_TOTAL', 'COST'], inplace=True)

del pfg_inv_fil
del pfg
'''

"    \npfg = pd.read_csv('inventory/pfg.txt', sep='\t', encoding_errors='ignore', escapechar='\\', low_memory=False, usecols=['SKU', 'STOCK_TOTAL', 'COST'] )\npfg['WD'] = 'J'\npfg.set_index( keys=['WD', 'SKU'], inplace=True)\n\npw = pw.join(other=pfg, on=['WD', 'Part Number'], how='left')   #Joining the price weight report with PFG inventory\n\npfg_inv_fil = (pw['WD']=='J') & (~(pw['STOCK_TOTAL'].isnull()))   #creating filter for items found in PFG inventory\n\n#Updating Stock and Cost from PFG inventory into the price weight report\npw.loc[pfg_inv_fil, 'MinPrice'] =pw.loc[pfg_inv_fil, 'COST']\npw.loc[pfg_inv_fil, 'item_cost'] =pw.loc[pfg_inv_fil, 'COST']\npw.loc[pfg_inv_fil, 'Qty'] =pw.loc[pfg_inv_fil, 'STOCK_TOTAL']\n\npw.loc[(pw['WD']=='J') & (pw['STOCK_TOTAL'].isnull()), 'Qty'] = 0   #Setting the stock of PFG items not in PFG inventory to 0\npw.drop(columns=['STOCK_TOTAL', 'COST'], inplace=True)\n\ndel pfg_inv_fil\ndel pfg\n"

#### Dorman Manual Price

In [13]:
'''
dprice = pd.read_csv('inventory/dorman_price.csv', low_memory=False)
dprice['WD'] = 'D'
dprice['COST'] = dprice['Cost'] + dprice['Core']
dprice.drop(columns=['Cost', 'Core'], inplace=True)
dprice['PN'] = dprice['PN'].map(lambda s: ''.join(filter(str.isalnum, s)))
dprice.set_index(['WD', 'LC', 'PN'], inplace=True)

pw = pw.join(dprice, on=['WD', 'LC', 'Part Number'], how='left')   #Joining pw-all with Dorman Price File

pw.loc[~pw['COST'].isnull(), 'MinPrice'] = pw.loc[~pw['COST'].isnull(), 'COST']  #Updating Prices from Dorman Price File
pw.loc[~pw['COST'].isnull(), 'item_cost'] = pw.loc[~pw['COST'].isnull(), 'COST']
pw.drop(columns='COST', inplace=True)
del dprice
'''

"\ndprice = pd.read_csv('inventory/dorman_price.csv', low_memory=False)\ndprice['WD'] = 'D'\ndprice['COST'] = dprice['Cost'] + dprice['Core']\ndprice.drop(columns=['Cost', 'Core'], inplace=True)\ndprice['PN'] = dprice['PN'].map(lambda s: ''.join(filter(str.isalnum, s)))\ndprice.set_index(['WD', 'LC', 'PN'], inplace=True)\n\npw = pw.join(dprice, on=['WD', 'LC', 'Part Number'], how='left')   #Joining pw-all with Dorman Price File\n\npw.loc[~pw['COST'].isnull(), 'MinPrice'] = pw.loc[~pw['COST'].isnull(), 'COST']  #Updating Prices from Dorman Price File\npw.loc[~pw['COST'].isnull(), 'item_cost'] = pw.loc[~pw['COST'].isnull(), 'COST']\npw.drop(columns='COST', inplace=True)\ndel dprice\n"

#### Dorman Manual Quantity

In [14]:
'''
dstock = pd.read_csv('inventory/dorman_stock.csv', low_memory=False)
dstock.columns.values[0] = 'PN'   #Renaming the Part Number column to PN
dstock.columns.values[1] = 'Stock'   #Renaming the Stock column
dstock['WD'] = 'D'
dstock['LC'] = 'DOR'
dstock = dstock[['WD', 'LC', 'PN', 'Stock']]  #Keepin gonly the required columns

dstock['PN'] = dstock['PN'].map(lambda s: ''.join(filter(str.isalnum, s)))   #Removing punctuation from Part Number
dstock.drop_duplicates(subset=['PN'], inplace=True)   #Removing the duplicate parts
dstock.set_index(['WD', 'LC', 'PN'], inplace=True)
#Applying stock definitions
dstock.loc[dstock['Stock']=='In Stock', 'Qty'] = 5
dstock.loc[dstock['Stock']=='Low Stock', 'Qty'] = 2
dstock.loc[dstock['Stock']=='Out of Stock', 'Qty'] = 0
dstock['Qty'] = dstock['Qty'].astype('int')
dstock.drop(columns='Stock', inplace=True)

pw = pw.join(dstock, on=['WD', 'LC', 'Part Number'], how='left', rsuffix='_d')   #Joining pw-all with Dorman Price File
pw.loc[~pw['Qty_d'].isna(), 'Qty'] = pw.loc[~pw['Qty_d'].isna(), 'Qty_d']   #Setting the Quantity from Dorman inventory to pw report
pw.drop(columns='Qty_d', inplace=True)   #Removing the inventory column
del dstock   #Removing the dorman inventory data frame as it is no longer needed
'''

"\ndstock = pd.read_csv('inventory/dorman_stock.csv', low_memory=False)\ndstock.columns.values[0] = 'PN'   #Renaming the Part Number column to PN\ndstock.columns.values[1] = 'Stock'   #Renaming the Stock column\ndstock['WD'] = 'D'\ndstock['LC'] = 'DOR'\ndstock = dstock[['WD', 'LC', 'PN', 'Stock']]  #Keepin gonly the required columns\n\ndstock['PN'] = dstock['PN'].map(lambda s: ''.join(filter(str.isalnum, s)))   #Removing punctuation from Part Number\ndstock.drop_duplicates(subset=['PN'], inplace=True)   #Removing the duplicate parts\ndstock.set_index(['WD', 'LC', 'PN'], inplace=True)\n#Applying stock definitions\ndstock.loc[dstock['Stock']=='In Stock', 'Qty'] = 5\ndstock.loc[dstock['Stock']=='Low Stock', 'Qty'] = 2\ndstock.loc[dstock['Stock']=='Out of Stock', 'Qty'] = 0\ndstock['Qty'] = dstock['Qty'].astype('int')\ndstock.drop(columns='Stock', inplace=True)\n\npw = pw.join(dstock, on=['WD', 'LC', 'Part Number'], how='left', rsuffix='_d')   #Joining pw-all with Dorman Price File\npw.loc

Manual Update verification

In [15]:
'''
tmp = pw.merge (pw1, on=['WD', 'LC', 'Part Number'], suffixes = ('_n', '_p' ) )
print (tmp.query("MinPrice_n != MinPrice_p").groupby('WD').size())
print (tmp.query("Qty_n != Qty_p").groupby('WD').size())
del tmp
#del pw1
'''

'\ntmp = pw.merge (pw1, on=[\'WD\', \'LC\', \'Part Number\'], suffixes = (\'_n\', \'_p\' ) )\nprint (tmp.query("MinPrice_n != MinPrice_p").groupby(\'WD\').size())\nprint (tmp.query("Qty_n != Qty_p").groupby(\'WD\').size())\ndel tmp\n#del pw1\n'

#### Calculate shipping by warehouse.

Read Shipping Data from file\
create column 'warehouse_key' in this dataframe by fetching key using the 'get_warehouse_key' function
Filter to include entries with Quantity >0 - exclude Quantities=0\
Converting 'Ship Cost' to per item
Filtering records with Shipping Cost > 0.05 and only considering a few columns

Reading price-file-shipping.csv and converting it all to a single dictionary | Using a series with unique index should give us the same performance

#### get_historical_shipping_estimates(df, warehouse_key):
    Filter the main (total) shipping data to the provided warehouse, join/merge df to this filtered shipping data based on SKU creating 'ship_weights'. Filter the records with weights between 0 and 1000 (removing garbage).
    If some clean data is available after this filtering, fit a LinearRegression() model on it to prodict Ship Cost based on dimensions and weights of the items. Displaying the RMS error of this trained model - Train, Test split missing here
    
##### Shipping cost prediction:
    if the sku is available in the recent price-file-shipping, use that shipping cost,
    if their is an order history of the sku, use the mean of the shipping cost of all historical orders
    if the shipping cost (LinearRegression) model exists for the warehouse (i.e. clean data is avaialable), predict the shipping cost using this model
    if their is some historical data available for that warehouse, use its mean
    else use 10 as shipping cost
    
#### calculate_warehouse_shipping(df, warehouse):
Expects a price/ weight dataframe format filtered for the warehouse. Returns with shipping price altered.
Assigns shipping values for each warehouse as per its rules.\
##### pfg
Checks the pfg inventory file to check for shipping and handling costs of the items. Sums these two costs and adds $3.5 to account for taxes. There should be some other way then this loop. (A simple join would probably be better for performance). If the item isn't found in the inventory file, use PFG_DEFAULT_SHIPPING (15)

##### Keystone
KEYSTONE_BASE_SHIPPING=11, KEYSTONE_LTL_SHIPPING=125

##### NPW
Possible overwrite of AC Delco SKUs (ignoring the 35)

In [16]:
shipping_data = pd.read_csv('shipping_data.csv', low_memory=False, parse_dates=['Ship Date'])
shipping_data['warehouse_key'] = shipping_data['Warehouse'].map(get_warehouse_key)
shipping_data = shipping_data[shipping_data['Quantity'] > 0]
shipping_data['Ship Cost'] = shipping_data['Ship Cost'] / shipping_data['Quantity']
shipping_data = shipping_data[shipping_data['Ship Cost'] > 0.05][['CS-SKU','warehouse_key','Ship Cost','Ship Date']]

price_file_shipping = pd.read_csv('price-file-shipping.csv', index_col='CS-SKU-NP')   #Optimized code instead of loop for dictionary conversion

In [17]:
def get_historical_shipping_estimates(df, warehouse_key): # (cssku, warehouse_key):    
    warehouse_shipping_data = shipping_data[(shipping_data['warehouse_key']==warehouse_key)].copy()

    # create warehouse-level shipping model
    feature_cols = ['Weight', 'DimWeight', 'ShipWeight', 'Length', 'Width', 'Height']
    ship_weights = df[['MasterSKU']+feature_cols].merge(warehouse_shipping_data, how='inner',
                                                        left_on='MasterSKU', right_on='CS-SKU').copy().dropna()
    ship_weights = ship_weights[(ship_weights['ShipWeight'] > 0) & (ship_weights['ShipWeight'] < 1000)]  
    if len(ship_weights) > 0:
        model = LinearRegression().fit(ship_weights[feature_cols], ship_weights['Ship Cost'])

         # log model error for audit purposes
        print('RMSE:',mean_squared_error(ship_weights['Ship Cost'], 
                                          model.predict(ship_weights[feature_cols]), squared=False))
    else:
         model = None


    #Getting Shipping price from recent price file
    df['cssku'] = df['WD']+df['MasterSKU']   #Creating cssku to fetch data from recent price file
    df = df.join(other=price_file_shipping, on='cssku', how='left', rsuffix='_pfs')
    df['Shipping'] = df['Shipping_pfs']
    df.drop(columns=['cssku', 'Shipping_pfs'], inplace=True)

    #Getting Mean of Shipping from Historical Shipping Data
    h_ship_avg = warehouse_shipping_data.groupby('CS-SKU').mean()['Ship Cost']
    df = df.join(other=h_ship_avg, how='left', on='MasterSKU')
    df.loc[df['Shipping'].isna(), 'Shipping'] = df.loc[df['Shipping'].isna(), 'Ship Cost']
    df.drop(columns=['Ship Cost'], inplace=True)

    #Getting Shipping Prediction from Dimensional LinearRegression model
    if len(ship_weights) > 0:
        df['ship_model'] = model.predict(df[feature_cols].fillna(0))
        df.loc[df['Shipping'].isna(), 'Shipping'] = df.loc[df['Shipping'].isna(), 'ship_model']
        df.drop(columns=['ship_model'], inplace=True)

    
    #Getting Warehouse Shipping Mean
    h_warehouse_avg = warehouse_shipping_data.groupby('warehouse_key').mean()['Ship Cost']
    df = df.join(other=h_warehouse_avg, on='WD')
    df.loc[df['Shipping'].isna(), 'Shipping'] = df.loc[df['Shipping'].isna(), 'Ship Cost']
    df.drop(columns=['Ship Cost'], inplace=True)

    #Baseline value of 10
    df.loc[df['Shipping'].isna(), 'Shipping'] = 10
    
    return df['Shipping'].values

# Expects something formatted like a price/weight DF, filtered for a warehouse
# returns the price/weight DF with shipping altered
def calculate_warehouse_shipping(df, warehouse):

    df = df.copy()
    
    print(warehouse)
    if warehouse=='D': # Dorman        
        #df.loc[(df['item_cost'] <= 30), ['Shipping', 'ShipMkup']] = 6, DEFAULT_SHIP_MARKUP # flat rate
        #df.loc[(df['item_cost'] > 30), ['Shipping', 'ShipMkup']] = 0, 1
        df.loc[:, ['Shipping', 'ShipMkup']] = 0, 1
        #df['ShipMkup'] = 1
        
    elif warehouse=='C': # Brock
        df.loc[(df['item_cost'] <= 50), 'Shipping'] = 12 # estimate / avg.
        df.loc[(df['item_cost'] > 50), 'Shipping'] = 0
        df['ShipMkup'] = 1 / 1.1 # to account for returns not being accepted
    elif warehouse=='P': # Parts Auth
        df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
        '''
        with open(PARTS_AUTH_SHIPPING_MODEL, 'rb') as f:
            m = pickle.load(f)
        df['lwh'] = df['Length'] * df['Width'] * df['Height']
        df['Shipping'] = m.predict(df[['Weight','Length','Width','Height','lwh']].fillna(0))
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
        '''
    elif warehouse=='1': # Jante
        df['Shipping'] = 0
        df['ShipMkup'] = 1
    elif warehouse=='J': # PFG        
        #Reading Shipping and Handling Cost from PGF inventory file and using their sum as Shipping Cost
        PFG_DEFAULT_SHIPPING = 15
        inv = pd.read_csv('inventory/pfg.txt', sep='\t', encoding_errors='ignore', escapechar='\\', low_memory=False, usecols=['SKU','SHIPPING_COST','HANDLING_COST'], index_col='SKU' )
        inv['pfg_cost'] = inv[['SHIPPING_COST', 'HANDLING_COST']].sum(1)
        inv.drop(columns=['SHIPPING_COST','HANDLING_COST'], inplace=True)
        df = df.join(other=inv, on='Part Number', how='left')
        df.loc[df['WD']=='J', 'pfg_cost'].fillna(PFG_DEFAULT_SHIPPING, inplace=True)
        df.loc[df['WD']=='J', 'Shipping'] = df.loc[df['WD']=='J', 'pfg_cost']
        df.drop(columns='pfg_cost', inplace=True)

        # ADD AN EXTRA $3.50 TO ACCOUNT FOR TAX MESS
        #df['Shipping'] = df['Shipping'] + 3.50 
        
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    elif warehouse=='K': # Keystone
        KEYSTONE_BASE_SHIPPING = 11.0
        KEYSTONE_LTL_SHIPPING = 175.0
        inv = pd.read_csv('inventory/keystone.csv', low_memory=False)
        # Since inventory file open, manage duplicate part # issue in Keystone by matching with UPC
        inv['PartNumber'] = inv['PartNumber'].str.replace('=','').str.replace('"','')
        ##inv['KeystoneShipping'] = (inv['UPS_Ground_Assessorial'] + KEYSTONE_BASE_SHIPPING).fillna(0)
        inv['KeystoneShipping'] = (KEYSTONE_BASE_SHIPPING)
        inv.loc[inv['UPSable']==False, 'KeystoneShipping'] = KEYSTONE_LTL_SHIPPING
        inv = inv.sort_values('KeystoneShipping', ascending=False).drop_duplicates(subset=['VendorCode','PartNumber'])
        df = df.merge(inv[['VendorCode','PartNumber','KeystoneShipping']], 
                      how='left', left_on=['LC','Part Number'], right_on=['VendorCode','PartNumber'])
        df['Shipping'] = df['KeystoneShipping']
        df['ShipMkup'] = 1
        df.drop(columns=['VendorCode', 'PartNumber', 'KeystoneShipping'], inplace=True)

    elif warehouse=='6': # Burco Mirrors
        df['Shipping'] = 8 # estimate / avg        
        df['ShipMkup'] = 1
    elif warehouse=='A': # APW
        df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    elif warehouse=='2': # OE Wheels
        df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
        
    elif warehouse=='5': # KSI Trading
        dgt = pd.read_csv('DGTire Shipping.csv', low_memory=False, dtype={'Part Number':'str'})   #Reading the DGTire Shipping Estimates from disk
        dgt['WD'] = '5'
        dgt['MasterLC'] = 150
        df = df.merge(dgt, how='inner', on=['WD', 'MasterLC', 'Part Number'], suffixes=('','_dgt'))   #Merging the pw-report dataframe with the Shipping Estimates dataframe
        df.loc[~df['ship_cost'].isna(), 'Shipping'] = df['ship_cost']   #Moving Shipping cost estimate to the Shipping column
        df.drop(columns=['Brand', 'ship_cost', 'shipping_est', 'FET'], inplace=True)   #Removing additional columns
        del dgt   #Deleting the wtd dataframe as it is no longer needed
    
    elif warehouse=='7': # WTD Arroyo
        wtd = pd.read_csv('WTD Shipping.csv', low_memory=False)   #Reading the WTD Shipping Estimates from disk
        wtd['WD'] = '7'
        wtd['MasterLC'] = 153
        df = df.merge(wtd, how='inner', on=['WD', 'MasterLC', 'Part Number'], suffixes=('','_wtd'))   #Merging the pw-report dataframe with the Shipping Estimates dataframe
        df.loc[~df['ship_cost'].isna(), 'Shipping'] = df['ship_cost']   #Moving Shipping cost estimate to the Shipping column
        df.drop(columns=['Brand', 'ship_cost'], inplace=True)   #Removing additional columns
        del wtd   #Deleting the wtd dataframe as it is no longer needed
        
    elif warehouse=='8': # Race Sport Lighting
        ##df['Shipping'] = df['Weight'].map(lambda w: 15 if (pd.isna(w) or w >= 1) else 6)
        ##df['ShipMkup'] = DEFAULT_SHIP_MARKUP        
        df['Shipping'] = 23.5
        df['ShipMkup'] = 1        
    elif warehouse=='9': # Sunbelt APG
        df['Shipping'] = 22
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    
    elif warehouse=='N': # NPW
        df['Shipping'] = 6
        df.loc[df['MinPrice']>15, 'Shipping'] = 10
        df.loc[df['MinPrice']>30, 'Shipping'] = 15
        df.loc[df['MinPrice']>50, 'Shipping'] = 20
        df.loc[df['MinPrice']>80, 'Shipping'] = 35
        df.loc[df['MinPrice']>150, 'Shipping'] = 60
        df.loc[df['MinPrice']>300, 'Shipping'] = 100
        df.loc[df['MinPrice']>500, 'Shipping'] = 200
        df['ShipMkup'] = DEFAULT_SHIP_MARKUP
    
    elif warehouse=='O': # Tonsa
        #df['Shipping'] = get_historical_shipping_estimates(df, warehouse)
        #df['ShipMkup'] = DEFAULT_SHIP_MARKUP
        df_cols = df.columns
        tonsa = pd.read_csv('Tonsa Shipping.csv', low_memory=False)
        tonsa['Part Number'] = tonsa['Tonsa#'].map(lambda s: ''.join(filter(str.isalnum, s)))   #Removing Punctuation from Part Number
        tonsa['WD'] = 'O'        
        df = df.merge(tonsa, how='left', left_on=['WD', 'LC', 'Part Number'], right_on=['WD', 'Mfg', 'Part Number'], suffixes=('','_tonsa'))
        df.loc[~df['ship_cost'].isna(), 'Shipping'] = df.loc[~df['ship_cost'].isna(), 'ship_cost']
        df = df[df_cols]
        del tonsa
        del df_cols

        
    elif warehouse=='Y': # MotorState
        df.loc[(df['item_cost'] <= 39.99), 'Shipping'] = 13
        df.loc[((df['item_cost'] > 39.99) & (df['item_cost'] <= 99.99)), 'Shipping'] = 12
        df.loc[(df['item_cost'] >= 100), 'Shipping'] = 11
        df['ShipMkup'] = 1        
    
    elif warehouse=='Z': # SimpleTire
        df['Shipping'] = 0        
        df['ShipMkup'] = 1    
        df_cols = df.columns
        st_FET = pd.read_csv('SimpleTire FET.csv', dtype={'PRODUCT_ID':str})   #Reading the FET file
        st_FET['WD'] = 'Z'
        df = df.merge(st_FET, left_on=['Part Number', 'WD'], right_on=['PRODUCT_ID', 'WD'], how='left')   #Merging with dataframe
        df.loc[~df['FET'].isna(), 'Shipping'] = df.loc[~df['FET'].isna(), 'FET']   #Copying FET as Shipping where applicable
        df = df[df_cols]   #Removing extra columns
        del df_cols, st_FET   #Deleting the dataframes which are no longer needed
    
    elif warehouse=='F':   #Keystone Crash (LKQ)
        LKQ = pd.read_csv('LKQ Shipping.csv', low_memory=False, usecols=['Product', 'Ship Cost'])   #Reading LKQ Ship Cost file from disk
        LKQ['WD'] = 'F'   
        df = df.merge(LKQ, how='left', left_on=['WD', 'Part Number'], right_on=['WD', 'Product'])   #Merging Price Weight Report with LKQ Ship Cost file
        df.loc[df['WD'] == 'F', 'Shipping'] = df.loc[df['WD'] == 'F', 'Ship Cost']
        df.drop (columns=['Product', 'Ship Cost'], inplace=True)
        del LKQ   #Deleting the LKQ Ship cost file from memory as it is no longer needed
        
    elif warehouse=='U': #Turn 14
        df['Shipping'] = 13
        
        #Using log function for prices above $5
        # a = 30.090392996378466
        # b = 1.5403837582252213
        # #a*math.log(i*b)
        # df['lship'] = df['MinPrice'].apply(lambda x:a*math.log(x*b) if x>5 else None )
        # df.loc[df['MinPrice']>5, 'Shipping'] = df['lship']
        # df.drop(columns=['lship'], inplace=True)
        
        df.loc[df['MinPrice']>15, 'Shipping'] = 15
        df.loc[df['MinPrice']>30, 'Shipping'] = 25
        df.loc[df['MinPrice']>50, 'Shipping'] = 20
        df.loc[df['MinPrice']>80, 'Shipping'] = 35
        df.loc[df['MinPrice']>150, 'Shipping'] = 60
        df.loc[df['MinPrice']>300, 'Shipping'] = 100
        df.loc[df['MinPrice']>500, 'Shipping'] = 180
        
        #Additional adjustments
        t14 = pd.read_csv('inventory/t14.csv', low_memory=False)  #Reading the Turn14 inventory file
        t14.columns.values[0] = 'sku'
        t14['part'] = t14.iloc[:,0].str.split('|', expand=True).iloc[:,1]   #Extracting part number from sku
        t14 = t14[['line_code', 'part', 'ltl_freight_required']].drop_duplicates()   #Removing extra columns
        df = df.merge(t14,  how='left', left_on=['LC', 'Part Number'], right_on=['line_code', 'part'])
        df.loc[df['ltl_freight_required']==1, 'Shipping'] = 175
        df.drop(columns='ltl_freight_required', inplace=True)
        del t14
        
        t14_DS = pd.read_csv('Turn 14 DS Detail.csv', low_memory=False)
        t14_DS['IPartNumber'] = t14_DS['IPartNumber'].str.lower()
        t14_DS['IPartNumber'] = t14_DS['IPartNumber'].str.replace(' ', '', regex=False)   #The Part in pw report doesn't have spaces
        df['t14_pn'] = df['LC'].str.lower() + df['Part Number'].str.lower()

        #Data adjustments for merge
        df.loc[df['LC'] == 'KW', 't14_pn'] = 'kws' + df.loc[df['LC'] == 'KW', 'Part Number'].str.lower()   #LC KW is changed to KWS 
        df.loc[df['LC'] == 'ATI', 't14_pn'] = 'app' + df.loc[df['LC'] == 'ATI', 'Part Number'].str.lower()   #ATI is different in file
        df.loc[df['LC'] == 'KPT', 't14_pn'] = 'kot' + df.loc[df['LC'] == 'KPT', 'Part Number'].str.lower()   #LC KPT is changed to KOT
        df.loc[((df['LC'] == 'NEX') & (df['Part Number'].str.startswith('SNO'))), 't14_pn'] = 'sno'+ df.loc[((df['LC'] == 'NEX') & (df['Part Number'].str.startswith('SNO'))), 'Part Number'].str.lower()
        df.loc[((df['LC'] == 'PER') & (df['Part Number'].str.startswith('AMP'))), 't14_pn'] = 'alt'+ df.loc[((df['LC'] == 'PER') & (df['Part Number'].str.startswith('AMP'))), 'Part Number'].str.lower()
        df.loc[((df['LC'].isna()) & (df['MasterLC']== 821)), 't14_pn'] = df.loc[((df['LC'].isna()) & (df['MasterLC']== 821)), 'Part Number'].str.lower()
        df.loc[((df['LC'].isna()) & (df['MasterLC']== 119)), 't14_pn'] = 'per' + df.loc[((df['LC'].isna()) & (df['MasterLC']== 119)), 'Part Number'].str.lower()

        df = df.merge( t14_DS, how='left', left_on='t14_pn', right_on='IPartNumber', suffixes=('', '_y'))

        df.loc[~df['DSFee'].isnull(), 'Shipping_Add'] = df.loc[~df['DSFee'].isnull(), 'Shipping_Add'] + df.loc[~df['DSFee'].isnull(), 'DSFee']   #Adding additional Drop Ship Fee as additional Shipping
        df = df.query("DropShip != 'never' ")   #Excluding the items which are never allowed to be drop shipped
        df.drop(columns=['t14_pn', 'DSFee', 'IPartNumber', 'DropShip', 'Weight_y'], inplace=True)
        del t14_DS
    
    elif warehouse=='3':        
        wp = pd.read_csv('WP_Shipping.csv', low_memory=False)
        wp['WD'] = '3'
        wp['Part Number'] = wp['Part Number'].map(lambda s: ''.join(filter(str.isalnum, s)))   #Removing Punctuation from sku
        wp.drop_duplicates(subset=['Part Number'], inplace=True)
        df = df.merge(wp, how='left', left_on=['LC', 'WD', 'Part Number'], right_on=['MasterLC', 'WD', 'Part Number'], suffixes=('','_wp'))
        df.loc[~df['ship_cost'].isnull(), 'Shipping'] = df.loc[~df['ship_cost'].isnull(), 'ship_cost']

        df.drop(columns=['ship_cost', 'Category', 'LC_wp', 'MasterLC_wp'], inplace=True)
        del wp
        
    elif warehouse=='4':
        df.loc[df['WD']=='4', 'Shipping'] = 0
        
    else:
        pass
    df['ShipMkup'] = df['ShipMkup'].fillna(1)
    return df

In [18]:
warnings.filterwarnings(action='ignore')

In [19]:
#new
dfs = []
for warehouse in pw['WD'].unique().tolist():
    wdf = pw[pw['WD']==warehouse].copy()
    wdfp = calculate_warehouse_shipping(wdf, warehouse)
                                                                           
    print('Proportion of parts missing shipping:', wdfp['Shipping'].isna().mean())
    dfs.append(wdfp)
    
pw = pd.concat(dfs, ignore_index=True)
#pw['ShipMkup'] = pw['ShipMkup'].round(3)

D
Proportion of parts missing shipping: 0.0
1
Proportion of parts missing shipping: 0.0
2
Proportion of parts missing shipping: 0.0
3
Proportion of parts missing shipping: 0.29829798730244494
4
Proportion of parts missing shipping: 0.0
5
Proportion of parts missing shipping: 0.0
6
Proportion of parts missing shipping: 0.0
7
Proportion of parts missing shipping: 0.0018673306064378445
8
Proportion of parts missing shipping: 0.0
9
Proportion of parts missing shipping: 0.0
K
Proportion of parts missing shipping: 0.12040887962303086
F
Proportion of parts missing shipping: 0.007897696998401515
Y
Proportion of parts missing shipping: 0.0
N
Proportion of parts missing shipping: 0.0
P
RMSE: 5.532038491377326
Proportion of parts missing shipping: 0.0
J
Proportion of parts missing shipping: 0.012560261149663835
Z
Proportion of parts missing shipping: 0.0
O
Proportion of parts missing shipping: 0.14407512665266278
U
Proportion of parts missing shipping: 0.0


In [20]:
#Deleting datasets from memory which are no longer needed
del shipping_data, price_file_shipping

#### Set inventory constraints

In [21]:
pw.loc[:, ['MinQty','MaxQty']] = 3, 12   ## Changed Min to two from three

pw.loc[pw['MasterLC']==145, ['MinQty','MaxQty']] = 10, 24   ## Updating min and Max Qty for Simple tire

#### Set price file defaults.

In [22]:
pw['ListMkup'] = .65
pw['SourceQty'] = None
pw['Source'] = None
pw['BundleSK'] = None
pw['Carrier'] = 'FedEx'
pw['Service'] = 'GroundHD'

Set OP-Lowest(Y) and VND-Lowest(Y) to "N" for Dorman

In [23]:
pw.loc[(pw['CS-SKU-NP'].str[0]=='D'), ['OP-Lowest(Y)','VND-Lowest(Y)'] ] = "N", "N"

## Quick fix for shipping

Where shipping is current zero for Tonsa, set it to 20. And 15 for Sunbelt.

In [24]:
pw.loc[((pw['Shipping'] == 0) & (pw['CS-SKU-NP'].str[0] == 'O')), 'Shipping'] = 20
pw.loc[((pw['Shipping'] == 0) & (pw['CS-SKU-NP'].str[0] == '9')), 'Shipping'] = 15

1.5x ~Tonsa, Sunbelt~, and Parts Auth shipping.

In [25]:
# pw.loc[(pw['CS-SKU-NP'].str[0].isin(['O','9','P'])), 'Shipping'] *= 1.5
pw.loc[(pw['CS-SKU-NP'].str[0]=='P'), 'Shipping'] *= 1.5

Double Eagle Eye Shipping.

In [26]:
pw.loc[(pw['CS-SKU-NP'].str[:4]=='P754'), 'Shipping'] *= 2.0

Bumper is expensive to ship.

In [27]:
pw.loc[pw['CS-SKU-NP']=='429|6448-0006', 'Shipping'] = 30

Another expensive shipping update from order: PSA669874628

In [28]:
pw.loc[pw['CS-SKU-NP']=='551|S6585B', 'Shipping'] = 46

Expensive shipping for part from 12/20/2021

In [29]:
pw.loc[pw['CS-SKU-NP']=='P576|3292', 'Shipping'] = 90

Expensive shipping for part from 12/27/2021

In [30]:
pw.loc[pw['CS-SKU-NP']=='P550|290073', 'Shipping'] = 100

In [31]:
pw.loc[pw['CS-SKU-NP']=='P308|55621', 'Shipping'] = 161

In [32]:
pw.loc[pw['CS-SKU-NP']=='P557|277504', 'Shipping'] = 35

In [33]:
pw.loc[pw['CS-SKU-NP']=='P643|ESK5752', 'Shipping'] = 34

In [34]:
pw.loc[pw['CS-SKU-NP']=='P551|40722A', 'Shipping'] = 7

In [35]:
pw.loc[pw['CS-SKU-NP']=='N643|AR8265XPR', 'PackQty'] = 1

In [36]:
pw.loc[pw['CS-SKU-NP']=='P123|33660', 'PackQty'] = 1

PA Shipping Costs from Umer analysis:

In [37]:
pa_shipping_data = pd.read_csv('PA Shipping Costs.csv', low_memory=False)
pa_shipping_data['WD'] = 'P'

pa_shipping_data = pa_shipping_data.sort_values(by='Row Labels', ascending=False)
pa_shipping_data.drop_duplicates(subset='Row Labels', keep="first")

pw = pw.merge(pa_shipping_data, how='left', left_on=['MasterSKU', 'WD'], right_on=['Row Labels', 'WD'])
pw.loc[pw['Final Shipping Cost'] > 0, 'Shipping'] = pw['Final Shipping Cost']
pw = pw[pw_cols]   #Restoring columns before this Merge

del pa_shipping_data   #removing variables no longer needed

NPW Pack Corrections and updates from Abdullah

In [38]:
pw.loc[pw['CS-SKU-NP']=='N223|97469', 'PackQty'] = 10
pw.loc[pw['CS-SKU-NP']=='N178|VL1093', 'PackQty'] = 4
pw.loc[pw['CS-SKU-NP']=='N223|98288', 'PackQty'] = 10 #PackQty issue - 20-May-2022
pw.loc[pw['CS-SKU-NP']== 'N114|3025', 'PackQty'] = 25

In [39]:
pw['MasterSKUn'] = pw['CS-SKU-NP'].str[1:]
shipping_fix = pd.read_csv('Shipping Corrections.csv', low_memory=False, encoding='unicode_escape')
shipping_fix = shipping_fix.drop_duplicates(['WD', 'SKU'])

pw = pw.merge(shipping_fix, how='left', left_on=['MasterSKUn', 'WD'], right_on=['SKU', 'WD'])
pw.loc[pw['Final Shipping Cost'] > 0, 'Shipping'] = pw['Final Shipping Cost']
pw.loc[pw['Shipping_Add']!=0.0, 'Shipping'] = pw.loc[pw['Shipping_Add']!=0.0, 'Shipping'] + pw.loc[pw['Shipping_Add']!=0.0, 'Shipping_Add']
pw = pw[pw_cols]

del shipping_fix   #removing variables no longer needed

In [40]:
new_shipping = pd.read_csv('newshipcosts.csv', low_memory=False)

pw = pw.merge(new_shipping, how='left', left_on=['MasterSKU'], right_on=['CS_SKU'])
pw.loc[pw['NewShipCost'] > 0, 'Shipping'] = pw['NewShipCost']
pw = pw[pw_cols]

del new_shipping   #removing variables no longer needed

Set MinQty really high for First Stop Brakes (Dorman line) to avoid actually selling any.

In [41]:
first_stop_brakes = pd.read_excel('8-2 Change 56 Brake Dropship and Stocking.xlsx', 
                                  skiprows=2, sheet_name='Dropship Price').rename(columns={'MATERIAL':'pn'})['pn']
pw.loc[(pw['WD']=='D') & pw['Part Number'].isin(first_stop_brakes), ['MinQty','MaxQty']] = 100, 100

del first_stop_brakes   #removing variables no longer needed

Handle RSL skus for MAP > Calculated Cost

In [42]:
#Reading RSL Inventory and Removing Duplicate SKUs
rsl_inventory = pd.read_csv('inventory/rsl.csv', low_memory=False)
rsl_inventory.sort_values(by='MAP', ascending=False, inplace=True)
rsl_inventory.drop_duplicates(subset=['SKU'], inplace=True, keep='first')
rsl_inventory['CS-SKU-NP'] = '8329|' + rsl_inventory['SKU']
rsl_inventory.set_index('CS-SKU-NP', inplace=True)


pw = pw.join(other = rsl_inventory, on='CS-SKU-NP', rsuffix='_rsl')
pw['tmp_mkup'] = pw['item_cost'] / ( (1 + 0.05) * (pw['item_cost'] + pw['Shipping']) / (1 - 0.15) - pw['Shipping'])
pw['tmp_price'] = pw['item_cost']/pw['tmp_mkup'] + pw['Shipping']/pw['ShipMkup']

pw.loc[pw['tmp_price']<pw['MAP_rsl'], ['MinPrice', 'item_cost']] = pw.loc[pw['tmp_price']<pw['MAP_rsl'], 'MAP_rsl']
pw.loc[pw['tmp_price']<pw['MAP_rsl'], ['Shipping', 'ShipMkup']] = 0

pw = pw[pw_cols]   #Restoring columns
del rsl_inventory   #removing variables no longer needed

In [43]:
motorstate = pd.read_csv('inventory/motorstate.csv', low_memory=False)
#motorstate['PartNumber'] = motorstate.PartNumber.str[3:]
motorstate['PartNumber'] = 'Y' + motorstate['PartNumber']
#pw = pw.merge(motorstate, how='left', left_on=['Part Number', 'WD'], right_on=['PartNumber', 'WD'])

pw['tkey'] = pw['WD'] + pw['LC'] + pw['Part Number']

pw = pw.merge(motorstate, how='left', left_on=['tkey'], right_on=['PartNumber'])
pw.drop(columns='tkey', inplace = True)

del motorstate   #removing variables no longer needed

In [44]:
pw.drop(pw.loc[pw['AirRestricted'] == 'YES'].index, inplace=True)
pw.drop(pw.loc[pw['TruckFrtOnly'] == 'YES'].index, inplace=True)

In [45]:
#pw.drop(pw.loc[(pw['WD'] == 'Y') & (pw['MasterLC'] != 261)].index, inplace=True)
pw = pw [pw_cols]   #Restoring Columns

NPW Min Order Qty

In [46]:
'''
npw_packqty = pd.read_csv('NPW PackQty.csv', low_memory=False)
npw_packqty['WD'] = 'N'
npw_packqty.set_index(['WD', 'Line Code', 'Part Number'], inplace=True)
npw_packqty

pw1 = pw.join(other=npw_packqty, on=['WD', 'LC', 'Part Number'], how='left')
pw1.loc[~pw1['NPW_PackQty'].isna()]

f = (pw1['WD']=='N') & (~pw1['NPW_PackQty'].isna())   #Creating the filter for items to be updated
pw1.loc[f & (pw1['PackQty']!=pw1['NPW_PackQty'])].to_excel('NPW PackQty Not Matching.xlsx')
'''

"\nnpw_packqty = pd.read_csv('NPW PackQty.csv', low_memory=False)\nnpw_packqty['WD'] = 'N'\nnpw_packqty.set_index(['WD', 'Line Code', 'Part Number'], inplace=True)\nnpw_packqty\n\npw1 = pw.join(other=npw_packqty, on=['WD', 'LC', 'Part Number'], how='left')\npw1.loc[~pw1['NPW_PackQty'].isna()]\n\nf = (pw1['WD']=='N') & (~pw1['NPW_PackQty'].isna())   #Creating the filter for items to be updated\npw1.loc[f & (pw1['PackQty']!=pw1['NPW_PackQty'])].to_excel('NPW PackQty Not Matching.xlsx')\n"

OE wheels price MAP fix 11/09

In [47]:
##oe_wheel_update = pd.read_excel('OE wheel price increase11-9.xlsx', sheet_name='Sheet1')
oe_wheel_update = pd.read_excel('OE Wheel Shipping.xlsx', sheet_name='Sheet1', dtype={'UPC':str})
oe_wheel_update.drop_duplicates(subset=['UPC'], inplace=True, keep='last')

oe_wheel_update['CS-SKU-NP'] = '2387|' + oe_wheel_update['UPC']
oe_wheel_update.set_index('CS-SKU-NP', inplace=True)

pw = pw.join(other = oe_wheel_update, on='CS-SKU-NP', rsuffix='_oe')
pw.loc[~pw['Shipping (Est)'].isna(), 'Shipping'] = pw.loc[~pw['Shipping (Est)'].isna(), 'Shipping (Est)']
pw = pw[pw_cols]   #Restoring Columns
del oe_wheel_update   #removing variables no longer needed

PA packQty Correction from inventory

In [48]:
pa_inv = pd.read_csv('inventory/pa.csv', low_memory=False, encoding= 'unicode_escape')
pa_inv['Part'] = pa_inv['Part'].str.replace('-', '', regex=False )
pa_inv['Part'] = pa_inv['Part'].str.replace('.', '', regex=False )
pa_inv['Part'] = pa_inv['Part'].str.replace('/', '', regex=False )
pa_inv['Total_Stock'] = pa_inv[['BxStock', 'ByStock', 'NYStock', 'DCStock', 'AZStock', 'CAStock', 'GAStock']].sum(axis=1)


pw = pw.merge(pa_inv, how='left', left_on=['Part Number', 'LC'], right_on=['Part', 'Line'], suffixes=('', 'y'))
pw.loc[~pw['Packs'].isna(), 'PackQty'] = pw.loc[~pw['Packs'].isna(), 'Packs']

#Manual Fetching the Price and Quantity from PA inventory
#pw.loc[~pw['Price'].isna(), 'MinPrice'] = pw.loc[~pw['Price'].isna(), 'Price']
#pw.loc[~pw['Price'].isna(), 'item_cost'] = pw.loc[~pw['Price'].isna(), 'Price']
#pw.loc[~pw['Total_Stock'].isna(), 'Qty'] = pw.loc[~pw['Total_Stock'].isna(), 'Total_Stock']

pw = pw[pw_cols]   #Restoring Columns

del pa_inv   #removing variables no longer needed

Force shipping cost and ship markup for RSL Skus

In [49]:
pw.loc[(pw['WD'] == '8') & (pw['Shipping'] == 0), ['Shipping', 'ShipMkup']] = 18, 1

In [50]:
#Removing Speicfic SKU(s) from Price Files
#SKU 1 P650|W41029 : Marion said on 30th September its Refunded 6 times so remove from price file 
pw = pw.loc[pw['CS-SKU-NP']!= "P650|W41029"]

In [51]:
#Preventing Race Sport Lightning items from any other warehouse
pw.drop(index=pw.query("MasterLC == 329 and WD != '8' ").index, inplace=True)

Exclude CoverCraft Temporarily (We are on their don't sell list)

In [52]:
pw = pw.loc[~pw['MasterLC'].isin ([571])]

Route Dorman Drive Shafts and Dorman Pack Items to Dorman only (remove it from any other warehouse)

In [53]:
DOI = pd.read_csv('Dorman Only Items.csv', low_memory=False, usecols=['MasterSKU']).drop_duplicates()   #Loading the List of items which should only be routed to Dorman
pw = pw.merge(right = DOI, how='left', on='MasterSKU', indicator=True)   #Merging the Dorman list to the Price Weight Report
pw.drop(pw.query("_merge == 'both' and WD!='D'").index, inplace=True)   #Dropping the DormanDriveShafts and Dorman Pack Items from any other Warehouse Except Dorman
pw.drop(columns = '_merge', inplace=True)   #Drop the merging indicator column
del DOI   #Delete the list as it is no longer needed

#### Calculate markups and format/write price files, by channel.

Load Pack SKUs Data from file (to be removed from Price Files)

In [54]:
packskus = pd.read_csv('Pack SKUs removed.csv', index_col='MasterSKU')

NPW Temporary min Ship Estimate set at 12 (for safe transition to non-flatrate shipping)

In [55]:
pw.loc[(pw['WD']=='N') & (pw['Shipping']<12), 'Shipping'] = 12

In [56]:
conf = pd.read_excel('configuration.xlsx', sheet_name=None )

In [57]:
conf['Channels'].set_index(keys = conf['Channels']['id'], inplace=True, verify_integrity=True, drop=False)   #Set the channel id as index
conf['Channels']['Catalog Fee'] = conf['Channels']['Catalog Fee'].fillna(0)   #Filling null Catalog Fee as 0
conf['Channels'] = conf['Channels'].to_dict('index')   #Converting to an easily searchable dictionary

LKQ Remove Paint bumper items

In [58]:
lkq_bp = pd.read_csv('LKQ Bumper Paint Issue.csv', low_memory=False)
pw = pw.merge(lkq_bp, left_on='MasterSKU', right_on='sku', how='left', indicator=True)
di = pw.query("_merge=='both'").index
pw.drop(index=di, inplace=True)
pw.drop(columns='_merge', inplace=True)

In [59]:
#Set the values from configuration table
def set_conf_val(dset, c_tbl, dsetcol, vcol): 
    for _, crow in c_tbl.iterrows():   #iterating over the configuration table entries
        crow = crow[lambda x: ~x.isna()] ###Excluding null columns
        
        if vcol in crow:   #if the Value column has a value in configuration

            def fil_fn(df):
                Trues = np.repeat(a=[True], repeats=len(df))            
                if 'Warehouse' in crow:
                    wd = df['WD'] == crow['Warehouse']
                else:                
                    wd = Trues.copy()

                if 'MasterLC' in crow:
                    lc = df['MasterLC'] == crow['MasterLC']
                else:                
                    lc = Trues.copy()                

                if 'Min Cost' in crow:
                    min_c = df['MinPrice'] >= crow['Min Cost']
                else:                
                    min_c = Trues.copy()

                if 'Max Cost' in crow:
                    max_c = df['MinPrice'] <= crow['Max Cost']
                else:                
                    max_c = Trues.copy()

                return ((wd) & (lc) & (min_c) & (max_c))


            dset.loc[fil_fn, dsetcol] = crow[vcol]   #Setting the dataset column with the provided value
            

In [60]:
#Exclude Warehouses and Brands from configuration file
def exclude_conf(dset, c_tbl):    
    for _, crow in c_tbl.iterrows():   # iterating over the configuration table entries
        crow = crow[lambda x: ~x.isna()] # Excluding null columns

        def fil_fn(df):
            Trues = np.repeat(a=[True], repeats=len(df))            
            if 'Warehouse' in crow:
                wd = df['WD'] == crow['Warehouse']
            else:                
                wd = Trues.copy()

            if 'MasterLC' in crow:
                lc = df['MasterLC'] == crow['MasterLC']
            else:                
                lc = Trues.copy()                

            if 'Min Cost' in crow:
                min_c = df['MinPrice'] >= crow['Min Cost']
            else:                
                min_c = Trues.copy()

            if 'Max Cost' in crow:
                max_c = df['MinPrice'] <= crow['Max Cost']
            else:                
                max_c = Trues.copy()

            return ((wd) & (lc) & (min_c) & (max_c))

        dset.drop( index= dset.loc[fil_fn].index, inplace=True)   #Excluding the items in the filter list
            

In [61]:
def process_channel(channel): 
    
    pf = pw.copy()
    pf['TP'] = 0.999    #Creating Target Profit column with huge placeholder value
    pf['STP'] = 0.999    #Creating Shipping Target Profit column with huge placeholder value
    pf['CAdd'] = 0;   #Creating Cost addition/adjustment column  with 0 default value
    
    c = conf['Channels'][channel['id']]   #Reading Channel info of corresponding channel from the configuration Excel
    channel_fee = c['Fee']     #Reading fee of the channel
    
    #Setting values from Configuration file
    set_conf_val(pf, conf['Target Profit'].query("Channel ==  @c['id'] or Channel.isna()"), 'TP', 'Value')   #Setting Target Profit as per configuration
    set_conf_val(pf, conf['Shipping Target Profit'].query("Channel ==  @c['id'] or Channel.isna()"), 'STP', 'Value')   #Setting Shipping Target Profit as per configuration
    set_conf_val(pf, conf['Quantities'].query("Channel ==  @c['id'] or Channel.isna()"), 'MinQty', 'MinQty')   #Setting MinQty as per configuration
    set_conf_val(pf, conf['Quantities'].query("Channel ==  @c['id'] or Channel.isna()"), 'MaxQty', 'MaxQty')   #Setting MaxQty as per configuration
    set_conf_val(pf, conf['Cost Addition'].query("Channel == @c['id'] or Channel.isna()"), 'CAdd', 'Value')   #Setting the Cost Addition value as per configuration

    pf['channel_fee'] = channel_fee + CS_TRANSACTION_FEE
    
    #Cleaning motorstate data - only retaining ALLSTARPERFORMANCE(261) or SKUs that don't contain punctuation and non-zero price
    pf['fsku'] = pf['CS-SKU-NP'].str[1:]
    pf['WC'] = pf.groupby(['MasterSKU'])['WD'].transform('nunique')   #Adding column for count of Warehouses in which each MasterSKU is present
    m = pf.query(" WD=='Y' and MasterLC!= [261, 412, 461, 404, 425, 342, 132, 414, 337, 233, 481, 850, 867, 479, 690, 579] and (fsku!=MasterSKU  or MinPrice==0 or WC==1)").index
    pf.drop(index=m, inplace=True)
        
    #Cleaning up Turn 14 - only keeping the SKUs without punctuation or the SKUs unique to Turn 14
    m = pf.query("WD == 'U' and (fsku!=MasterSKU or WC==1)").index
    pf.drop(index=m, inplace=True)
    
    pf.drop(inplace=True, columns='WC')
    
    exclude_conf(pf, conf['Exclusions'].query("Channel == @c['id'] or Channel.isna()"))   #Excluding warehouses and Brands


    if 'Ebay' in channel['name']:    #Removing Ebay Violation SKUs from Price File    
        #Removing the Holley Carburetor Repair Kit and Magnaflow Exhaust System Kits
        eBay_Violation_skus = pd.read_csv('EBay Violation SKUs.csv', low_memory = False, usecols=['SKU'])        
        pf = pf.merge(eBay_Violation_skus, how='left', left_on='MasterSKU', right_on='SKU')
        pf = pf.loc[pf['SKU'].isnull()]
        pf.drop(columns = 'SKU', inplace = True)

    pf['CatSKU'] = 'N'      #Treat all SKUs as non-Cat by default
    #Filtering to Master List and populating CatSKU column
    if not (pd.isna(c['MasterFile'])):
        m_file = pd.read_csv(c['MasterFile'], dtype=str, low_memory = False)   #Reading the Master File
        
        sku_col = 'MasterSKU'
        #Joining with the Master file based on whether the WD column is present or not        
        if 'MasterSKU' not in m_file.columns:   #Determining whether to use master sku or final sku to join
            sku_col = 'SKU'
            pf['SKU'] = pf['CS-SKU-NP'].str[1:]   #Extracting final sku by removing the warehouse key from the CS-SKU-NP
            
        if 'WD' in m_file.columns:   
            pf = pf.merge(right=m_file, how='inner', on = ['WD', sku_col], suffixes=('', '_m') )
        else:
            pf = pf.merge(right=m_file, how='inner', on = sku_col, suffixes=('', '_m') )
        
        if 'CatSKU_m' in pf.columns:
            pf.loc[~pf['CatSKU_m'].isna(), 'CatSKU'] = pf.loc[~pf['CatSKU_m'].isna(), 'CatSKU_m']   #Populating the CatSKU column from the Master data
            pf.drop(columns='CatSKU_m', inplace=True)   #Dropping the extra column        
            pf.loc[pf['CatSKU']=='Y', 'CS-SKU-NP'] = pf.loc[pf['CatSKU']=='Y', 'WD-SKU']
            pf.loc[pf['CatSKU']=='Y', 'channel_fee'] = pf.loc[pf['CatSKU']=='Y', 'channel_fee'] + c['Catalog Fee']   #Adding Catalog Fee to Catalog SKUs Channel fee
        del m_file   #Delete Master File from memory as it is no longer needed
        
 
        
        
    if channel['name'] != 'AP Fusion':
        #Removing SKUs with Pack Quantities (except Dorman Pack SKUs)
        pf = pf.merge(right = packskus, on='MasterSKU', how='left', indicator=True)
        pf.drop ( index = pf.loc[pf['_merge']=='both'].index, columns='_merge', inplace=True)
    else:
        pf.loc[pf['MinPrice']<20.12, 'MinPrice'] = 20.12   #Setting minimum output price of an item in APFusion to $20.12
        pf.loc[((pf['WD']=='F') & (pf['Shipping']>=180)), 'STP'] = 0.02   #Adjusting Shipping Target Profit

    #Markup Calculation
    pf['Markup'] = (1-pf['channel_fee']) / (1+pf['TP'])
    pf['ShipMkup'] = (1-pf['channel_fee']) / (1+pf['STP'])
    
    # pf.loc[pf['item_cost']<=50,'Markup'] = pf.loc[pf['item_cost']<=50,'Markup'] / (pf.loc[pf['item_cost']<=50,'item_cost'] + 4) * pf.loc[pf['item_cost']<=50,'item_cost'] ###Upping the morkup to map the original price
    CAddfil = ( (pf['CAdd']!=0) & (~pf['item_cost'].isna()) & (~pf['Markup'].isna()))   #Filter for cost Adjustments
    pf.loc[CAddfil, 'Markup'] = pf.loc[CAddfil, 'Markup'] / (pf.loc[CAddfil, 'item_cost'] + pf.loc[CAddfil, 'CAdd']) * pf.loc[CAddfil, 'item_cost']   #Applying the cost adjustments
    
    # Clean up
    pf = pf.loc[~pf['Markup'].isnull()]   #Removing items with null markup
    pf = pf.loc[~pf['Shipping'].isnull()]   #Removing items with no Shipping
    
    
    #Rounding
    pf.loc[pf['MinPrice'] < 1,'MinPrice'] = 1
    pf.loc[pf['Markup'] < .1,'Markup'] = .1
    pf.loc[pf['Markup'] > 1,'Markup'] = 1
    pf['Markup'] = pf['Markup'].round(3)
    pf['ShipMkup'] = pf['ShipMkup'].round(3)
    pf['Shipping'] = pf['Shipping'].round(2)
    pf.loc[pf['Shipping'].lt(0), 'Shipping'] = 0    
    
    pf['fprice'] = (pf['item_cost'] * pf['PackQty']) / pf['Markup'] + pf['Shipping']/pf['ShipMkup']   #Computing the Final Total cost - For Future Use
    

    #Making skus unique - i.e. routing each sku to only one warehouse
    pfl = []
    pfl.append(pf.query("WD == 'D' and Qty>0 "))   #Dorman preference - route any parts available in Dorman to Dorman
    pfl.append(pf.query("MasterLC == 308 and WD == 'K' and Qty>0").sort_values(['fprice', 'Qty'], ascending=[True, False]))   #Prefer Walker Exhaust to be routed through Keystone if available

    qty_threshold = 5           
    #Prefer Lowest final price for items with available quantity >= threshold (5)    (except NPW)
    pfl.append(pf.query("Qty >= @qty_threshold and WD!='N'") .sort_values(['fprice', 'Qty'], ascending=[True,False])\
           .drop_duplicates(subset=['MasterSKU'], keep='first'))

    #Putting in all NPW parts with Quantity >= threshold (5) so that NPW will be applicable if quantity in other warehouses is less than threshold
    pfl.append(pf.query("Qty >= @qty_threshold and WD=='N'"))


    #Prefer Highest available quantity for any items with available quantity below threshold
    pfl.append(pf.query("Qty>0 and Qty < @qty_threshold and WD!='N'").sort_values(['Qty','fprice'], ascending=[False,True])\
           .drop_duplicates(subset=['MasterSKU'], keep='first'))

    pfl.append(pf.query("Qty>0 and Qty < @qty_threshold and WD=='N'"))

    pfl.append(pf.query("Qty==0").sort_values(['Qty','fprice'], ascending=[False,True])\
       .drop_duplicates(subset=['MasterSKU'], keep='first'))

    pf = pd.concat(pfl)

    pf = pf.drop_duplicates (subset=['MasterSKU'], keep='first')  
    
        
    #Processing Bundle SKUs
    if not (pd.isna(c['VariantFile'])):
        b_file = pd.read_csv(c['VariantFile'], dtype = str, low_memory = False, index_col='MasterSKU')   #Reading the list of Variant/ Bundle SKUs
        
        #Getting unique variant skus and setting the duplicate flag to 'Y'
        unique_bundles = b_file.reset_index()['MasterSKU'].drop_duplicates()
        pf = pf.merge(unique_bundles, how='left', on='MasterSKU', indicator=True)

        #Setting The Duplicate indicator to 'Y' for the SKUs which are present in Bundle SKUs list
        pf.loc[pf['_merge']=='both', 'Duplicate'] = 'Y'
        pf.loc[pf['_merge']!='both', 'Duplicate'] = 'N'
        pf.drop(columns='_merge', inplace=True)

        #Preparing a list of Variants to be merged with main price file
        Bundles = pf.join(other=b_file, how='inner', on= 'MasterSKU')
        Bundles['BundleSKU'] = Bundles['CS-SKU-NP']
        Bundles['CS-SKU-NP'] = Bundles['V-SKU']

        #Adding the additional Shipping
        if 'Add_Ship' in b_file.columns:
            Bundles['Shipping'] = Bundles['Shipping'] + pd.to_numeric(Bundles['Add_Ship'].fillna(0))
            Bundles.drop(columns='Add_Ship', inplace=True)            
            
        Bundles.drop(columns='V-SKU', inplace=True)
        # Merging the Main Price File with the Variants
        pf = pd.concat([pf, Bundles], ignore_index=True)
        del b_file, unique_bundles, Bundles
        
    
        
    # Return the Price File
    return pf

In [62]:
sm = []   #For extraction of shipping Target Profits
for c in CHANNELS:
    t1 = time.time()
    pf = process_channel(c)   #Processing the channel and getting Price File
    
    s = pf.groupby(['TP', 'WD']).size().reset_index(name='count')
    s['name'] = c['name']
    sm.append(s)
    
    pf = pf[PRICE_FILE_COLUMNS]   #Only taking the columns needed for the Price File
    
    # Write price file to Disk
    pf.to_csv(f"{PRICE_FILE_LOCATION}/{c['name']}.csv", index=False)
    
    t2 = time.time()
    print (c, round(t2-t1,3))
    
end_time = time.time()

print ('Total Time Taken : ', end_time - start_time, ' seconds')

{'name': 'AP Fusion', 'id': 'APF'} 17.839
{'name': 'PS Amazon', 'id': 'PSA'} 25.382
{'name': 'PS Walmart', 'id': 'PSW'} 22.469
{'name': 'PS Ebay', 'id': 'PSE'} 30.714
{'name': 'BS Amazon', 'id': 'BSA'} 17.273
{'name': 'BS Walmart', 'id': 'BSW'} 20.255
{'name': 'BS Ebay', 'id': 'BSE'} 22.105
{'name': 'Mecka', 'id': 'BAABS'} 18.469
{'name': 'My Motor Madness', 'id': 'MMM'} 17.15
Total Time Taken :  285.6590905189514  seconds


In [87]:
print ('here')

here


In [80]:
pf = pw.query("PackQty == 1 and Qty>0 ")

In [81]:
pf['WC'] = pf.groupby(['MasterSKU'])['WD'].transform('nunique') 

In [82]:
pf['total_cost'] = pf['MinPrice'] + pf['Shipping']

In [83]:
pf['tcost_min'] = pf.groupby(['MasterSKU'])['total_cost'].transform('min') 
pf['tcost_max'] = pf.groupby(['MasterSKU'])['total_cost'].transform('max') 

pf['cost_min'] = pf.groupby(['MasterSKU'])['MinPrice'].transform('min')
pf['cost_max'] = pf.groupby(['MasterSKU'])['MinPrice'].transform('max') 

In [84]:
pf.sort_values(by='MinPrice', inplace=True)
'''
pf['pcost'] = pf.groupby(['MasterSKU'])['MinPrice'].shift(1)
pf['ncost'] = pf.groupby(['MasterSKU'])['MinPrice'].shift(-1)

pf['pcost_WD'] = pf.groupby(['MasterSKU'])['WD'].shift(1)
pf['ncost_WD'] = pf.groupby(['MasterSKU'])['WD'].shift(-1)

pf.sort_values(by='total_cost', inplace=True)
pf['ptcost'] = pf.groupby(['MasterSKU'])['total_cost'].shift(1)
pf['ntcost'] = pf.groupby(['MasterSKU'])['total_cost'].shift(-1)

pf['ptcost_WD'] = pf.groupby(['MasterSKU'])['WD'].shift(1)
pf['ntcost_WD'] = pf.groupby(['MasterSKU'])['WD'].shift(-1)
'''

"\npf['pcost'] = pf.groupby(['MasterSKU'])['MinPrice'].shift(1)\npf['ncost'] = pf.groupby(['MasterSKU'])['MinPrice'].shift(-1)\n\npf['pcost_WD'] = pf.groupby(['MasterSKU'])['WD'].shift(1)\npf['ncost_WD'] = pf.groupby(['MasterSKU'])['WD'].shift(-1)\n\npf.sort_values(by='total_cost', inplace=True)\npf['ptcost'] = pf.groupby(['MasterSKU'])['total_cost'].shift(1)\npf['ntcost'] = pf.groupby(['MasterSKU'])['total_cost'].shift(-1)\n\npf['ptcost_WD'] = pf.groupby(['MasterSKU'])['WD'].shift(1)\npf['ntcost_WD'] = pf.groupby(['MasterSKU'])['WD'].shift(-1)\n"

In [85]:
pf['cost_rank'] = pf.groupby(['MasterSKU'])['MinPrice'].rank(method='first')

In [86]:
pf['tcost_rank'] = pf.groupby(['MasterSKU'])['total_cost'].rank(method='first')

In [87]:
mstate = pf.query("WD == 'Y'")[['MasterSKU']].drop_duplicates()

In [88]:
mstate

,MasterSKU
390575,803|S5248AL
390542,803|GMCA424
390630,803|T5173BLK
409893,487|0050
359732,261|99164
...,...
388641,487|5272663
400724,620|52346S400
377728,487|1100210612R
400723,620|52343S400


In [89]:
pf = pf.merge(mstate, on='MasterSKU', how='inner')

In [90]:
pf[['WD', 'MasterLC', 'Part Number', 'MasterSKU', 'Qty', 'CS-SKU', 'MinPrice', 'Shipping', 'PackQty', 'fname', 'item_cost', 'CS-SKU-NP', 'WC', 'total_cost', 'tcost_min', 'tcost_max', 'cost_min', 'cost_max', 'cost_rank', 'tcost_rank'
                            ]].to_excel('Motorstate.xlsx')